In [94]:
#!pip install sentence-transformers

In [95]:
from sentence_transformers import SentenceTransformer, util
from PIL import Image, ImageFile
import glob
import torch
import pickle
import zipfile
from IPython.display import display
from IPython.display import Image as IPImage
import torch.nn.functional as F
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import pprint

In [101]:
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value

In [96]:
pp = pprint.PrettyPrinter(indent=4, width=200, depth=None, stream=None, compact=False, sort_dicts=False)

img_folder = 'images_30k/'

test_df = pd.read_csv(f"test_df_from_30k.csv")

captions_w_fnames = test_df.to_dict('records')

d_imgs_w_texts = {} 
for e in captions_w_fnames:
    append_value(d_imgs_w_texts, e['image'], e['caption'])

In [98]:
test_df

,image,caption_number,caption,id
0,1001545525.jpg,0,Két férfi Németországban egyszerre ugrott át a...,9
1,1001545525.jpg,1,Két fiatal ugrál át az út menti korláton éjszaka.,9
2,1001545525.jpg,2,Rúdon táncoló fiúk az éjszaka közepén.,9
3,1001545525.jpg,3,Két ing nélküli férfi ugrál át a sínen.,9
4,1001545525.jpg,4,két srác együtt ugrott át egy kapun,9
...,...,...,...,...
31780,99804383.jpg,0,"Egy idősebb, szemüveges busker egy keleti vonó...",31781
31781,99804383.jpg,1,Ez egy férfi egy bolt előtt fellép egy kisfiúv...,31781
31782,99804383.jpg,2,Egy idősebb ázsiai férfi hangszeren játszik eg...,31781
31783,99804383.jpg,3,Egy idős férfi ül a kirakat előtt egy fiatal f...,31781


In [128]:
# Now, we need to compute the embeddings
# To speed things up, we destribute pre-computed embeddings
# Otherwise you can also encode the images yourself.
# To encode an image, you can use the following code:
# from PIL import Image
# img_emb = model.encode(Image.open(filepath))

# Here we load the multilingual CLIP model. Note, this model can only encode text.
# If you need embeddings for images, you must load the 'clip-ViT-B-32' model
text_model = SentenceTransformer('clip-ViT-B-32-multilingual-v1')

#For embedding images, we need the non-multilingual CLIP model
img_model = SentenceTransformer('clip-ViT-B-32')

#img_names = list(glob.glob(f'{img_folder}*.jpg'))
test_imgs = list(d_imgs_w_texts.keys())

captions = list(test_df['caption'].values)

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


Images: 6357


In [ ]:
img_embs = np.empty([len(test_imgs), 512])
for idx, file in enumerate(test_imgs):
    img = Image.open(img_folder+file)
    print(idx,end='\r')
    img_embs[idx] = img_model.encode(img)

In [134]:
text_embeddings = text_model.encode(captions)

In [140]:
def find_matches_from_img_to_texts(text_embeddings, image_fn, captions, n=5):

    img = Image.open(img_folder+image_fn)
    image_embedding = img_model.encode(img)

    image_embedding_t = torch.from_numpy(image_embedding)
    text_embeddings_t = torch.from_numpy(text_embeddings)

    text_embeddings_n = F.normalize(text_embeddings_t, p=2, dim=-1)   # all caption 
    image_embedding_n = F.normalize(image_embedding_t, p=2, dim=-1)   # just one image

    dot_similarity = image_embedding_n @ text_embeddings_n.T

    values, indices = torch.topk(dot_similarity.squeeze(0), n)
    
    matches = [captions[idx] for idx in indices]

    return matches


In [144]:
img_cnt = len(d_imgs_w_texts)
caption_cnt = img_cnt*5

In [ ]:
cnt=0
res_sum = 0
cnt_when_not_zero=0

for k,v  in d_imgs_w_texts.items():
    #print(cnt)
    res=0
    # k = img_fname v = list_of_captions_for_the_img
    zs=find_matches_from_img_to_texts(text_embeddings, k, captions, n=5)
    #print("expected: ",k, v)
    #print("got: ", zs)
    for text in zs:
        if(text in v): res+=1
    #print(f'{res}/5')
    if res != 0 : cnt_when_not_zero+=1 
    res_sum += res
    cnt+=1

In [145]:
print(cnt_when_not_zero, '/', img_cnt)
print(res_sum ,'/', caption_cnt)

3370 / 6357
5779 / 31785


In [130]:
# it works the same way as the search method from huggingface
def find_matches_from_text_to_img(image_embeddings, caption, image_filenames, n=5):

    text_embedding = text_model.encode(caption)

    image_embeddings_t = torch.from_numpy(image_embeddings).float()
    text_embedding_t = torch.from_numpy(text_embedding).float()

    image_embeddings_n = F.normalize(image_embeddings_t, p=2, dim=-1)
    text_embedding_n = F.normalize(text_embedding_t, p=2, dim=-1)

    dot_similarity = text_embedding_n @ image_embeddings_n.T
    values, indices = torch.topk(dot_similarity.squeeze(0), n)
    print(values)
    print(indices)
    matches = [image_filenames[idx] for idx in indices]
    
    return matches

In [ ]:
find_matches_from_text_to_img(img_embs, "két srác együtt ugrott át egy kapun", test_imgs, n=5)